# 1. Setup and Imports

In [1]:
import sys
import warnings

import numpy as np
import pandas as pd

from pathlib import Path

# Add the project folder to sys.path
ROOT = Path.cwd().parent
sys.path.append(str(ROOT))

from notebooks.helpers.data_diagnostics import (
    check_duplicates,
    unique_and_dtype_summary
)

from sklearn.model_selection import train_test_split

In [2]:
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# 2. Load Data

In [3]:
file_path = Path.cwd().parent / "data" / "raw" / "flight_price.csv"

In [4]:
data = pd.read_csv(file_path)
print('Data Loaded')

Data Loaded


# 3. Overview of the Data
## 3.1. General Overview

In [5]:
print(f"Number of entries: {data.shape[0]}")
print(f"Number of features: {data.shape[1]}")

Number of entries: 10683
Number of features: 11


In [6]:
data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [7]:
data.tail()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648
10682,Air India,9/05/2019,Delhi,Cochin,DEL → GOI → BOM → COK,10:55,19:15,8h 20m,2 stops,No info,11753


## 3.2. Dataset Structure: Rows, Columns, Types, and Memory
- `Data Conformity` check

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


### **3.2. Observations & Action Plan**
- Data contains **10683** observations/entries and **11** features.
- `Price` is the **target** variable.
- Data features contain **10 object** datatypes and **1 int** datatype.
- Datatypes of some features are not appropriate.
- `Route` & `Total_Stops` features have a **missing value** each, which we will explore in next section.
- Feature names to be **lower** cased for consistency.

## 3.3. Check for Duplicate Entries
- `Data Uniqueness` check.

In [9]:
check_duplicates(data)

There are 220 duplicate entries in the data.


In [10]:
(
    data
    .loc[data.duplicated(keep=False)]
    .sort_values(
        by=['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route']
    )
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


In [11]:
display(
    data
    .loc[data.duplicated(), 'Airline']
    .value_counts()
    .to_frame()
    .rename(columns={'count': 'duplicates'})
    .assign(
        total_count=data['Airline'].value_counts(),
        unique_count=lambda x: x['total_count'] - x['duplicates'],
        airline_duplicate_pct=lambda x: round(x['duplicates']/x['total_count'] * 100, 2),
        overall_duplicate_pct=lambda x: round(x['duplicates']/data.shape[0]*100, 2),
        
    )
    .sort_values(by='airline_duplicate_pct', ascending=False)
)

,duplicates,total_count,unique_count,airline_duplicate_pct,overall_duplicate_pct
Airline,,,,,
Jet Airways,149,3849,3700,3.87,1.39
Air India,57,1752,1695,3.25,0.53
IndiGo,10,2053,2043,0.49,0.09
SpiceJet,3,818,815,0.37,0.03
Vistara,1,479,478,0.21,0.01


### **3.3. Observations & Action Plan**
- There are **220** duplicate entries in the data.
- **Jet Airways** has the most number of duplicates **(149)**.
- These duplicate entries to be **dropped**, to have balanced analysis.

## 3.4. Check for Data Correctness
- `Data Accuracy`, `Data Consistency`, `Data Validation` & `Data Relevancy` check

In [12]:
unique_and_dtype_summary(df=data)

,feature,unique_count,data_type
0,Price,1870,int64
1,Arrival_Time,1343,object
2,Duration,368,object
3,Dep_Time,222,object
4,Route,128,object
5,Date_of_Journey,44,object
6,Airline,12,object
7,Additional_Info,10,object
8,Destination,6,object
9,Source,5,object


### **`3.4.1. Price`**

In [13]:
data['Price']

0         3897
1         7662
2        13882
3         6218
4        13302
         ...  
10678     4107
10679     4145
10680     7229
10681    12648
10682    11753
Name: Price, Length: 10683, dtype: int64

In [14]:
data['Price'].describe()

count    10683.000000
mean      9087.064121
std       4611.359167
min       1759.000000
25%       5277.000000
50%       8372.000000
75%      12373.000000
max      79512.000000
Name: Price, dtype: float64

In [15]:
(
    data
    ['Price']
    .astype('str')
    .str
    .extract(r"([^0-9]++)", expand=False)
    .unique()
)

array([nan], dtype=object)

### **`3.4.1. Observations & Action Plan`**
- `Price` is **numeric** feature (Target variable).
- All the values seem valid and accurate, so **no** cleaning required.
- Feature name to be **lower cased** for consistency - `price`.

### **`3.4.2. Arrival_Time`**

In [16]:
data['Arrival_Time']

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [17]:
data['Arrival_Time'].unique()

array(['01:10 22 Mar', '13:15', '04:25 10 Jun', ..., '06:50 10 Mar',
       '00:05 19 Mar', '21:20 13 Mar'], shape=(1343,), dtype=object)

In [18]:
(
    data
    ['Arrival_Time']
    .loc[lambda x: x.str.contains(r"([^0-9:])")]
    .str.split(" ", n=1)
    .str[-1]
    .unique()
    
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### **`3.4.2. Observations & Action Plan`**
- `Arrival Time` is time in **24 hour** format, so its datatype to be changed to **datetime**.
- There are certain entries, where **date** & **month** is attached alongwith arrival time (For flights where arrival is next day).
- This effect of date change is already captured in `Duration` feature. So for consistency, values to be **fixed to keep only time (ex: 19:15)**.
- Feature name to be **lower cased** - `arrival_time`.

### **`3.4.3. Duration`**

In [19]:
data['Duration']

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [20]:
data['Duration'].unique()

array(['2h 50m', '7h 25m', '19h', '5h 25m', '4h 45m', '2h 25m', '15h 30m',
       '21h 5m', '25h 30m', '7h 50m', '13h 15m', '2h 35m', '2h 15m',
       '12h 10m', '26h 35m', '4h 30m', '22h 35m', '23h', '20h 35m',
       '5h 10m', '15h 20m', '2h 55m', '13h 20m', '15h 10m', '5h 45m',
       '5h 55m', '13h 25m', '22h', '5h 30m', '10h 25m', '5h 15m',
       '2h 30m', '6h 15m', '11h 55m', '11h 5m', '8h 30m', '22h 5m',
       '2h 45m', '12h', '16h 5m', '19h 55m', '3h 15m', '25h 20m', '3h',
       '16h 15m', '15h 5m', '6h 30m', '25h 5m', '12h 25m', '27h 20m',
       '10h 15m', '10h 30m', '1h 30m', '1h 25m', '26h 30m', '7h 20m',
       '13h 30m', '5h', '19h 5m', '14h 50m', '2h 40m', '22h 10m',
       '9h 35m', '10h', '21h 20m', '18h 45m', '12h 20m', '18h', '9h 15m',
       '17h 30m', '16h 35m', '12h 15m', '7h 30m', '24h', '8h 55m',
       '7h 10m', '14h 30m', '30h 20m', '15h', '12h 45m', '10h 10m',
       '15h 25m', '14h 5m', '20h 15m', '23h 10m', '18h 10m', '16h',
       '2h 20m', '8h', '16h 5

In [21]:
(
    data['Duration']
    .loc[lambda ser: ~ser.str.contains('m', na=False)]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [22]:
(
    data['Duration']
    .loc[lambda ser: ~ser.str.contains('h', na=False)]
    .unique()
)

array(['5m'], dtype=object)

In [23]:
(
    data['Duration']
    .loc[lambda ser: ~ser.str.contains('h')]
    .index
)

Index([6474], dtype='int64')

### **`3.4.3. Observations & Action Plan`**
- `Duration` feature gives the **duration** of flight.
- It has values in **mixed** format - **Hour & Minutes (Ex: 2h30m)**.
- For **consistency** in the feature values, we will convert duration in **minutes** and later change datatype to **numeric**.
- **Index 6474** has the duration as **5m**, which **doesn't** seem to be accurate. So that entry to be dropped to maintain valid and accurate data.
- Feature name to be **lower cased** - `duration`.

### **`3.4.4. Departure Time`**

In [24]:
data['Dep_Time']

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [25]:
data['Dep_Time'].unique()

array(['22:20', '05:50', '09:25', '18:05', '16:50', '09:00', '18:55',
       '08:00', '08:55', '11:25', '09:45', '20:20', '11:40', '21:10',
       '17:15', '16:40', '08:45', '14:00', '20:15', '16:00', '14:10',
       '22:00', '04:00', '21:25', '21:50', '07:00', '07:05', '09:50',
       '14:35', '10:35', '15:05', '14:15', '06:45', '20:55', '11:10',
       '05:45', '19:00', '23:05', '11:00', '09:35', '21:15', '23:55',
       '19:45', '08:50', '15:40', '06:05', '15:00', '13:55', '05:55',
       '13:20', '05:05', '06:25', '17:30', '08:20', '19:55', '06:30',
       '14:05', '02:00', '09:40', '08:25', '20:25', '13:15', '02:15',
       '16:55', '20:45', '05:15', '19:50', '20:00', '06:10', '19:30',
       '04:45', '12:55', '18:15', '17:20', '15:25', '23:00', '12:00',
       '14:45', '11:50', '11:30', '14:40', '19:10', '06:00', '23:30',
       '07:35', '13:05', '12:30', '15:10', '12:50', '18:25', '16:30',
       '00:40', '06:50', '13:00', '19:15', '01:30', '17:00', '10:00',
       '19:35', '15:

In [26]:
(
    data
    ['Dep_Time']
    .astype('str')
    .str
    .extract(r"([^0-9:]++)", expand=False)
    .unique()
)

array([nan], dtype=object)

### **`3.4.4. Observations & Action Plan`**
- Datatype of `Dep_Time` to be changed to **datetime**.
- Since format and values are valid, **no** initial cleaning steps required.
- Feature name to be **lower cased** - `dep_time`.

### **`3.4.5. Route`**

In [27]:
data['Route']

0                    BLR → DEL
1        CCU → IXR → BBI → BLR
2        DEL → LKO → BOM → COK
3              CCU → NAG → BLR
4              BLR → NAG → DEL
                 ...          
10678                CCU → BLR
10679                CCU → BLR
10680                BLR → DEL
10681                BLR → DEL
10682    DEL → GOI → BOM → COK
Name: Route, Length: 10683, dtype: object

In [28]:
data['Route'].unique()

array(['BLR → DEL', 'CCU → IXR → BBI → BLR', 'DEL → LKO → BOM → COK',
       'CCU → NAG → BLR', 'BLR → NAG → DEL', 'CCU → BLR',
       'BLR → BOM → DEL', 'DEL → BOM → COK', 'DEL → BLR → COK',
       'MAA → CCU', 'CCU → BOM → BLR', 'DEL → AMD → BOM → COK',
       'DEL → PNQ → COK', 'DEL → CCU → BOM → COK', 'BLR → COK → DEL',
       'DEL → IDR → BOM → COK', 'DEL → LKO → COK',
       'CCU → GAU → DEL → BLR', 'DEL → NAG → BOM → COK',
       'CCU → MAA → BLR', 'DEL → HYD → COK', 'CCU → HYD → BLR',
       'DEL → COK', 'CCU → DEL → BLR', 'BLR → BOM → AMD → DEL',
       'BOM → DEL → HYD', 'DEL → MAA → COK', 'BOM → HYD',
       'DEL → BHO → BOM → COK', 'DEL → JAI → BOM → COK',
       'DEL → ATQ → BOM → COK', 'DEL → JDH → BOM → COK',
       'CCU → BBI → BOM → BLR', 'BLR → MAA → DEL',
       'DEL → GOI → BOM → COK', 'DEL → BDQ → BOM → COK',
       'CCU → JAI → BOM → BLR', 'CCU → BBI → BLR', 'BLR → HYD → DEL',
       'DEL → TRV → COK', 'CCU → IXR → DEL → BLR',
       'DEL → IXU → BOM → COK', 'CCU 

In [29]:
(
    data
    ['Route']
    .astype('str')
    .str
    .extract(r"([^a-zA-Z]++)", expand=False)
    .unique()
)

array([' → ', nan], dtype=object)

### **`3.4.5. Observations & Action Plan`**
- No initial cleaning steps required.
- Feature name to be **lower cased** - `route`.

### **`3.4.6. Date of Journey`**

In [30]:
data['Date_of_Journey']

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [31]:
data['Date_of_Journey'].unique()

array(['24/03/2019', '1/05/2019', '9/06/2019', '12/05/2019', '01/03/2019',
       '24/06/2019', '12/03/2019', '27/05/2019', '1/06/2019',
       '18/04/2019', '9/05/2019', '24/04/2019', '3/03/2019', '15/04/2019',
       '12/06/2019', '6/03/2019', '21/03/2019', '3/04/2019', '6/05/2019',
       '15/05/2019', '18/06/2019', '15/06/2019', '6/04/2019',
       '18/05/2019', '27/06/2019', '21/05/2019', '06/03/2019',
       '3/06/2019', '15/03/2019', '3/05/2019', '9/03/2019', '6/06/2019',
       '24/05/2019', '09/03/2019', '1/04/2019', '21/04/2019',
       '21/06/2019', '27/03/2019', '18/03/2019', '12/04/2019',
       '9/04/2019', '1/03/2019', '03/03/2019', '27/04/2019'], dtype=object)

In [32]:
(
    data
    ['Date_of_Journey']
    .str
    .extract(r"([^0-9]++)", expand=False)
    .unique()
)

array(['/'], dtype=object)

### **`3.4.6. Observations & Action Plan`**
- `Date_of_Journey` reports date, month and year of the travel.
- Data is valid and maintains consistent formatting where **date**, **month** and **year** are separated by **`/`** (Ex: 27/04/2019).
- Datatype to be changed to **datetime**.
- Feature named to be **lower cased** - `date_of_journey`

### **`3.4.7. Airline`**

In [33]:
data['Airline']

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [34]:
data['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [35]:
(
    data
    ['Airline']
    .str
    .extract(r"([^a-zA-Z ]++)", expand=False)
    .unique()
)

array([nan], dtype=object)

### **`3.4.7. Observations & Action Plan`**
- We will merge **Jet Airways Business** into **Jet Airways**, **Vistara Premium economy** into **Vistara** and **Multiple carriers Premium economy** into **Multiple carriers**.
- Feature name to be **lower cased** - `airline`.

### **`3.4.8. Additional Information`**

In [36]:
data['Additional_Info']

0        No info
1        No info
2        No info
3        No info
4        No info
          ...   
10678    No info
10679    No info
10680    No info
10681    No info
10682    No info
Name: Additional_Info, Length: 10683, dtype: object

In [37]:
data['Additional_Info'].unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [38]:
(
    data
    ['Additional_Info']
    .astype('str')
    .str
    .extract(r"([^a-zA-Z0-9- ]++)", expand=False)
    .unique()
)

array([nan], dtype=object)

### **`3.4.8. Observations & Action Plan`**
- **No info** and **No Info** are treated separately due to case formatting.
- For clarity and to handle such issue, object feature values to be stripped and lower cased.
- Feature name to be **lower cased** as `additional_info`.

### **`3.4.9. Destination`**

In [39]:
(
    data
    ['Destination']
)

0        New Delhi
1         Banglore
2           Cochin
3         Banglore
4        New Delhi
           ...    
10678     Banglore
10679     Banglore
10680        Delhi
10681    New Delhi
10682       Cochin
Name: Destination, Length: 10683, dtype: object

In [40]:
data['Destination'].unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### **`3.4.9. Observations & Action Plan`**
- **Banglore** to be corrected as **Bengaluru**, **Cochin** as **Kochi** and **Delhi** to be merged with **New Delhi** to maintain correctness of city names.
- Feature name to be **lower cased** - `destination`.

### **`3.4.10. Source`**

In [41]:
data['Source']

0        Banglore
1         Kolkata
2           Delhi
3         Kolkata
4        Banglore
           ...   
10678     Kolkata
10679     Kolkata
10680    Banglore
10681    Banglore
10682       Delhi
Name: Source, Length: 10683, dtype: object

In [42]:
data['Source'].unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### **`3.4.10. Observations & Action Plan`**
- **Banglore** to be corrected as **Bengaluru** and **Delhi** to be merged with **New Delhi** to maintain correctness of city names.
- Feature name to be **lower cased** - `source`.

### **`3.4.11. Total Stops`**

In [43]:
data['Total_Stops']

0        non-stop
1         2 stops
2         2 stops
3          1 stop
4          1 stop
           ...   
10678    non-stop
10679    non-stop
10680    non-stop
10681    non-stop
10682     2 stops
Name: Total_Stops, Length: 10683, dtype: object

In [44]:
data['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

### **`3.4.11. Observations & Action Plan`**
- Map values to a numerical values **(non-stop - 0, 1 stop - 1, 2 stops - 2, 3 stops - 3, 4 stops - 4)**.
- Feature name to be **lower cased** - `total_stops`.

# 4. Data Cleaning
- Data cleaning steps will be performed as discussed in **section 3**

In [45]:
def extract_time(ser):
    return(
        ser
        .str.split(" ", expand=True).set_axis(['Hour','Minute'], axis=1)
        .assign(
            Hour=lambda df_: df_['Hour'].str.replace('h', '').astype('int').mul(60),
            Minute=lambda df_: df_['Minute'].str.replace('m', '').fillna('0').astype('int')
        )
        .sum(axis=1)
    )

In [46]:
def clean_data(data):
    return (
        data
        .drop_duplicates(keep='first')
        .drop(index=[6474])
        .replace({
            'Airline': {'Jet Airways Business': 'Jet Airways',
                        'Vistara Premium economy': 'Vistara',
                        'Multiple carriers Premium economy': 'Multiple carriers'},
            'Source': {'Banglore': 'Bengaluru',
                       'Delhi': 'New Delhi'},
            'Destination': {'Banglore': 'Bengaluru',
                            'Delhi': 'New Delhi',
                            'Cochin': 'Kochi'}
        })
        .apply(lambda x: (
            x.str.strip().str.lower()
            if x.dtype == 'object'
            else x
        ))
        .assign(
            Date_of_Journey=lambda df: (
                pd.to_datetime(df['Date_of_Journey'], dayfirst=True)
            ),
            
            Dep_Time=lambda df: (
                pd.to_datetime(df['Dep_Time'], dayfirst=True)),
            
            Arrival_Time=lambda df: (
                pd.to_datetime(df['Arrival_Time'].str.split(" ", n=1).str[0]).dt.time),
            
            Duration = lambda df: df['Duration'].pipe(extract_time),
            
            Total_Stops = lambda df: (
                df['Total_Stops']
                .map({'non-stop': 0, '1 stop': 1, '2 stops': 2, '3 stops': 3, '4 stops': 4})
            )
        )
        .rename(columns=str.lower)
    )

In [47]:
df_clean = clean_data(data)

print("Initial Data Cleaning Done!!")

Initial Data Cleaning Done!!


# 5. Clean Data Summary

## 5.1. General Overview

In [48]:
print(f"Number of entries: {df_clean.shape[0]}")
print(f"Number of features: {df_clean.shape[1]}")

Number of entries: 10462
Number of features: 11


In [49]:
df_clean.head()

,airline,date_of_journey,source,destination,route,dep_time,arrival_time,duration,total_stops,additional_info,price
0,indigo,2019-03-24,bengaluru,new delhi,blr → del,2026-02-13 22:20:00,01:10:00,170,0.0,no info,3897
1,air india,2019-05-01,kolkata,bengaluru,ccu → ixr → bbi → blr,2026-02-13 05:50:00,13:15:00,445,2.0,no info,7662
2,jet airways,2019-06-09,new delhi,kochi,del → lko → bom → cok,2026-02-13 09:25:00,04:25:00,1140,2.0,no info,13882
3,indigo,2019-05-12,kolkata,bengaluru,ccu → nag → blr,2026-02-13 18:05:00,23:30:00,325,1.0,no info,6218
4,indigo,2019-03-01,bengaluru,new delhi,blr → nag → del,2026-02-13 16:50:00,21:35:00,285,1.0,no info,13302


In [50]:
df_clean.tail()

,airline,date_of_journey,source,destination,route,dep_time,arrival_time,duration,total_stops,additional_info,price
10678,air asia,2019-04-09,kolkata,bengaluru,ccu → blr,2026-02-13 19:55:00,22:25:00,150,0.0,no info,4107
10679,air india,2019-04-27,kolkata,bengaluru,ccu → blr,2026-02-13 20:45:00,23:20:00,155,0.0,no info,4145
10680,jet airways,2019-04-27,bengaluru,new delhi,blr → del,2026-02-13 08:20:00,11:20:00,180,0.0,no info,7229
10681,vistara,2019-03-01,bengaluru,new delhi,blr → del,2026-02-13 11:30:00,14:10:00,160,0.0,no info,12648
10682,air india,2019-05-09,new delhi,kochi,del → goi → bom → cok,2026-02-13 10:55:00,19:15:00,500,2.0,no info,11753


## 5.2. Dataset Structure: Rows, Columns, Types, and Memory

In [51]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10462 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   airline          10462 non-null  object        
 1   date_of_journey  10462 non-null  datetime64[ns]
 2   source           10462 non-null  object        
 3   destination      10462 non-null  object        
 4   route            10461 non-null  object        
 5   dep_time         10462 non-null  datetime64[ns]
 6   arrival_time     10462 non-null  object        
 7   duration         10462 non-null  int64         
 8   total_stops      10461 non-null  float64       
 9   additional_info  10462 non-null  object        
 10  price            10462 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(2), object(6)
memory usage: 980.8+ KB


# 6. Train Test Split

In [52]:
train_df, test_df = train_test_split(df_clean, test_size=0.2, random_state=12)

In [53]:
print("Train Set:")
print(f"Number of entries: {train_df.shape[0]}")
print(f"Number of features: {train_df.shape[1]}")

print("-" * 40)

print("Test Set:")
print(f"Number of entries: {test_df.shape[0]}")
print(f"Number of features: {test_df.shape[1]}")

print("-" * 40)

Train Set:
Number of entries: 8369
Number of features: 11
----------------------------------------
Test Set:
Number of entries: 2093
Number of features: 11
----------------------------------------


In [54]:
display(train_df.head())

,airline,date_of_journey,source,destination,route,dep_time,arrival_time,duration,total_stops,additional_info,price
5298,spicejet,2019-06-09,kolkata,bengaluru,ccu → blr,2026-02-13 09:00:00,11:25:00,145,0.0,no info,4174
893,air asia,2019-05-15,kolkata,bengaluru,ccu → ixr → del → blr,2026-02-13 15:10:00,23:30:00,500,2.0,no info,5192
917,multiple carriers,2019-06-12,new delhi,kochi,del → bom → cok,2026-02-13 17:00:00,01:30:00,510,1.0,no info,7198
10498,jet airways,2019-05-06,kolkata,bengaluru,ccu → bom → blr,2026-02-13 16:30:00,09:20:00,1010,1.0,in-flight meal not included,9663
1737,indigo,2019-05-06,chennai,kolkata,maa → ccu,2026-02-13 07:55:00,10:15:00,140,0.0,no info,3850


In [55]:
display(test_df.head())

,airline,date_of_journey,source,destination,route,dep_time,arrival_time,duration,total_stops,additional_info,price
3974,air asia,2019-05-21,bengaluru,new delhi,blr → del,2026-02-13 11:10:00,13:55:00,165,0.0,no info,3383
116,air india,2019-03-06,new delhi,kochi,del → maa → cok,2026-02-13 17:20:00,09:25:00,965,1.0,no info,12677
1289,multiple carriers,2019-06-24,new delhi,kochi,del → bom → cok,2026-02-13 03:50:00,12:35:00,525,1.0,no info,12192
4253,indigo,2019-05-21,new delhi,kochi,del → bom → cok,2026-02-13 10:35:00,01:30:00,895,1.0,no info,6794
6963,indigo,2019-05-01,kolkata,bengaluru,ccu → bbi → blr,2026-02-13 19:35:00,23:15:00,220,1.0,no info,4226


# 7. Save and Export Split Data

In [56]:
save_file_dir = Path.cwd().parent / "data" / "processed"

train_file_path = save_file_dir / "cleaned_train.csv"
test_file_path = save_file_dir / "cleaned_test.csv"

save_file_dir.mkdir(parents=True, exist_ok=True)

train_df.to_csv(train_file_path, index=False)
train_df.to_csv(test_file_path, index=False)

print(f"Train set saved")
print(f"Test set saved")

Train set saved
Test set saved
